# Step 1. 의사소견에 대해 실시간 text 생성
- file 형식: ? - 실시간으로 입력해야 하는데...
- model: 'faster-whisper': https://github.com/SYSTRAN/faster-whisper
- language: 한국어, "ko"
- prompt: 선택사항. -> 선택않고 그냥 그대로 옮겨 적겠음
- response_format: text(or json?)
- temperature: 의사의 소견을 있는 그대로 옮기는 게 1순위이므로, temperature=0.0으로 지정해준다

(실시간으로 누군가(의사)가 말하는 문장의 positive와 negative, netural를 띄워주는 기능도 구현할 수 있는데 이러한 기능 또한 도입하면 좋을까? 고민중...)

In [2]:
# #토큰 정보로드를 위한 라이브러리
# from dotenv import load_dotenv
# #토큰 정보로드
# load_dotenv()

True

In [2]:
# import os
# import time
# from openai import OpenAI

# client = OpenAI()

In [6]:
# audio_file = open("C:\\Users\\parkh\\[WISET] source code\\04.OpenAI\\04.OpenAI\\data\\채용면접_샘플_01.wav", "rb")
# transcript = client.audio.transcriptions.create(
#     file=audio_file,
#     model="whisper-1",
#     language="ko",
#     response_format="text",
#     temperature=0.0,
# )

In [7]:
# #결과물 출력
# print(transcript)

지금 생각해보면 가장 기억에 남는 것이 외환위기 때의 경험입니다. 외환위기 때 나라뿐 아니라 회사에서는 달러가 많이 부족했고 우리는 수출을 하기 위해서 원자재라든지 대금지급 혹은 또 우리가 수출한 물건에 대한 대금을 받아야 되는 상황이었습니다. 일단은 우리가 해외로부터 받아야 될 때는 최대한 그것을 달러로 받았고 반대로 우리가 지불해야 될 것은 가능한 한 원자재를 통해서 지급을 했습니다. 그 원자재라는 것이 결국은 반도체 쪽이었는데 우리나라에서 가장 나름 손쉽게 구하면서도 꼭 필요한 제품인 반도체를 원자재값 대응으로 물건을 주면서 반대로 해외에서 우리가 받아야 될 것은 달러로 받으면서 그 환율차를 가장 줄일 수가 있었습니다. 그것으로 인해서 약 6개월 동안에 나름 회사에서는 달러 지출을 막을 수 있었고 그때 그나마 달러를 회사에서 확보를 해서 나름의 위기를 극복할 수 있었던 것으로 생각합니다.



In [9]:
# !pip install pyaudio numpy noisereduce openai-whisper

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 0.0/798.6 kB ? eta -:--:--
     --- ----------------------------------- 61.4/798.6 kB 1.6 MB/s eta 0:00:01
     ------ ------------------------------- 143.4/798.6 kB 1.7 MB/s eta 0:00:01
     ------------- ------------------------ 276.5/798.6 kB 2.1 MB/s eta 0:00:01
     --------------- ---------------------- 327.7/798.6 kB 2.3 MB/s eta 0:00:01
     ---------------------- --------------- 471.0/798.6 kB 2.0 MB/s eta 0:00:01
     ------------------------------- ------ 655.4/798.6 kB 2.3 MB/s eta 0:00:01
     -------------------------------------- 798.6/798.6 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject

DEPRECATION: celery 5.0.5 has a non-standard dependency specifier pytz>dev. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [6]:
# !pip install pyaudio numpy whisper noisereduce

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 0.0/42.8 kB ? eta -:--:--
     --------- ------------------------------ 10.2/42.8 kB ? eta -:--:--
     ------------------ ------------------- 20.5/42.8 kB 162.5 kB/s eta 0:00:01
     -------------------------------------- 42.8/42.8 kB 296.3 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41393 sha256=5401a431d35445ba892ca214ccca7551c123534bd1667f104ddd7859bd14ad47
  Stored in directory: C:\Users\parkh\AppData\Local\Temp\pip-ephem-wheel-cache-7g07etpq\wheels\21\65\ee\4e6672aabfa486d3341a39a04f8f87c77e5156149299b5a7d0
Successfully built whisper


DEPRECATION: celery 5.0.5 has a non-standard dependency specifier pytz>dev. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of celery or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [1]:
import pyaudio
import wave
import numpy as np
import whisper
import time
import os
import noisereduce as nr

# Whisper 모델 초기화
model = whisper.load_model("large")

# 파라미터 설정
FORMAT = pyaudio.paInt16  # 오디오 포맷
CHANNELS = 1              # 모노 오디오
RATE = 16000              # 샘플링 레이트
CHUNK = 1024              # 버퍼 크기
RECORD_SECONDS = 60       # 녹음 시간 (초)

# PyAudio 초기화
audio = pyaudio.PyAudio()

# 마이크 스트림 열기
stream = audio.open(format=FORMAT, channels=CHANNELS,
                    rate=RATE, input=True,
                    frames_per_buffer=CHUNK)

print("Recording...")

frames = []

# 녹음 시작
try:
    start_time = time.time()
    while time.time() - start_time < RECORD_SECONDS:
        data = stream.read(CHUNK)
        frames.append(np.frombuffer(data, dtype=np.int16))
    
    print("Recording stopped.")
except KeyboardInterrupt:
    print("Recording interrupted.")

# 프레임을 numpy 배열로 변환
audio_data = np.hstack(frames)

# 노이즈 감소 적용
def reduce_noise(audio_data, rate):
    reduced_noise = nr.reduce_noise(y=audio_data, sr=rate)
    return reduced_noise

clean_audio_data = reduce_noise(audio_data, RATE)

# 오디오 데이터를 WAV 파일로 저장
wav_path = "recorded_audio.wav"
with wave.open(wav_path, 'wb') as wf:
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(audio.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(clean_audio_data.tobytes())

# Whisper를 사용하여 오디오 변환
result = model.transcribe(wav_path, language="ko")
transcription = result["text"]

# 결과를 화면에 출력
print("Transcription:", transcription)

# 결과를 텍스트 파일로 저장
with open("transcription.txt", "w", encoding="utf-8") as file:
    file.write(transcription)

print("Transcription saved to 'transcription.txt'")

# 임시 파일 삭제
os.remove(wav_path)

# 스트림 닫기
stream.stop_stream()
stream.close()
audio.terminate()

C:\Users\parkh\anaconda3\envs\wiset\Lib\site-packages\whisper\__init__.py:65: UserWarning: C:\Users\parkh\.cache\whisper\large-v3.pt exists, but the SHA256 checksum does not match; re-downloading the file
  warnings.warn(
100%|█████████████████████████████████████| 2.88G/2.88G [24:13<00:00, 2.12MiB/s]


Recording...
Recording stopped.


C:\Users\parkh\anaconda3\envs\wiset\Lib\site-packages\whisper\transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription:  환자의 보험은 2차 소견을 구하는 특정한 보험을 사용하는 것입니다. 환자는 본인의 주치의에게 다른 의사가 전문의의 추천을 요청할 수 있습니다. 의사들은 대부분 다른 의견을 환영합니다. 그러나 두 번째 의사는 두 번째 의사와 동일한 관점을 공유할 수 있는 치밀한 관계자로 선임이 안됩니다. 담당 의사에게 요청한 곳에 마음 놓이지 않으나 환자는 본인이 신뢰하는 다른 의사에게 부탁할 수 있습니다. 그렇지 않다면 흔히 대학병원이나 전문의료협회 또는 보험회사에서 환자의 보험을 사용하는 것입니다.
Transcription saved to 'transcription.txt'


In [4]:
#생성된 treanscription.txt파일을 JSON형식으로도 저장하는 코드
import json

with open("transcription.txt", "w", encoding="utf-8") as file:
    file.write(transcription)
print("Transcription saved to 'transcription.txt'")

#결과를 JSON 파일로 저장
json_data = {
    "transcription": transcription
}
with open("transcription.json", "w", encoding="utf-8") as json_file:
    json.dump(json_data, json_file, ensure_ascii=False, indent=4)
print("Transcription saved to 'transcription.json'")

Transcription saved to 'transcription.txt'
Transcription saved to 'transcription.json'
